In [4]:
# import modules
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import pandas as pd
import ethnicolr as ec
from pathlib import Path
import string
from time import sleep
import string
import re

# test queries
test = pd.concat(map(pd.read_csv, [str(Path().resolve().parents[2]) + "\Reference\Indian-Female-Names.csv",
                                 str(Path().resolve().parents[2]) + "\Reference\Indian-Male-Names.csv"]))
for alpha in string.ascii_lowercase:
    tmp = pd.read_csv(str(Path().resolve().parents[2]) + "\Reference\Indian-Surnames.csv")
    tmp['first name'] = alpha
    test = test.append(tmp, ignore_index=True)
test = test.drop_duplicates()
test = test.dropna()
test = test.reset_index(drop=True)

# dictionary to hold data
data = {
    'First Name' : [],
    'Last Name' : [],
    'Email' : []
}

# unwanted prefixes
prefixes = ["President", "Prof", "Dr", "Dean", "Dr."]

# open browser and go to directory
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-extensions')
driver = str(Path().resolve().parents[2]) + "\Tools\chromedriver.exe"
browser = webdriver.Chrome(executable_path=driver, options=chrome_options)
browser.get('http://directory.utoronto.ca/phonebook/pages/admin/main.xhtml')

# iterate through all queries
for i in test.index:
    # make next query
    f = test.at[i, 'first name']
    l = test.at[i, 'last name']
    # find first name, last name fields and search button
    lname_field = browser.find_element_by_id('searchFormPerson:lastName')
    fname_field = browser.find_element_by_id('searchFormPerson:firstName')
    search_button = browser.find_element_by_id('searchFormPerson:personSearchButton')
    lname_field.clear()
    fname_field.clear()
    lname_field.send_keys(l)
    fname_field.send_keys(f) 
    search_button.click()

    # check number of results
    results = browser.find_elements_by_css_selector("table[id^='SearchResultsForm:personResults:'")

    # if results are found
    if results != []:
        # iterate through each person
        for prof in results:
            # obtain contact info and clean it
            full_name = prof.find_element_by_class_name('name-item-header').text.title().split()
            if full_name[0] in prefixes:
                del full_name[0]
            fname = full_name[0]
            lname = full_name[-1]
            email = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", prof.get_attribute('innerHTML'), re.I)
            if email != []:
                email = email[0]
            else:
                email = "No Email"

            # store contact info in data dictionary
            data['First Name'] += [fname]
            data['Last Name'] += [lname]
            data['Email'] += [email]
                
            # print contact info
            print(fname, lname, email)
                
# close browser
browser.quit()

# create pandas dataframe and remove any duplicate entries, add manually retrieved contact info
df = pd.DataFrame(data)
tmp = pd.read_csv(str(Path().resolve().parents[1]) + "\Manual Entry\Database Expansion.csv")
tmp[["First Name", "Last Name"]] = tmp["Name"].str.split(" ", n = 1, expand = True) 
tmp.drop(columns =["Name"], inplace = True)
tmp = tmp[['First Name', 'Last Name', 'Email']]
df.append(tmp, ignore_index=True)
df = df.drop_duplicates()
df = df.dropna()
df = df.reset_index(drop=True)
    
# determine ethnicity
df = ec.pred_wiki_name(df,list(df)[1],list(df)[0])
    
# print dataframe and write data to csv file
print(df)
df.to_csv("uoft parsed.csv")

Aisha Ahmad aisha.ahmad@utoronto.ca
Aisha Ahmad aisha.ahmad@utoronto.ca
Nisha Andany nisha.andany@mail.utoronto.ca
Anisha Bishun abishun@utschools.ca
Nisha Fernandes nfernandes@msh.on.ca
Nisha Kanani Nisha.Kanani@thp.ca
Disha Kapoor disha.kapoor@utoronto.ca
Vishal Kukreti Vishal.Kukreti@uhn.ca
Vishal Kukreti vishal.kukreti@uhn.ca
Maisha Louis maisha.louis@utoronto.ca
Aisha Mirza meng.admission@mie.utoronto.ca
Nisha Mistry nisha.mistry@utoronto.ca
Telisha Mullings telisha.mullings@utoronto.ca
Nisha Panchal integrityadmin@utsc.utoronto.ca
Vishal Patel vishal.patel@utoronto.ca
Kenisha Peters kenishapeters@hotmail.com
Nisha Ravindran nisha.ravindran@utoronto.ca
Misha Schwartz mschwa@cs.toronto.edu
Aisha Sheikh aisha.sheikh@utoronto.ca
Telisha Smith-Gorvie telisha.smith-gorvie@uhn.ca
Mansik Hovanessian mansik.hovanessian@utoronto.ca
Pooja Malhotra pooja.malhotra@utoronto.ca
Tehmeena Jadoon tehmeena.jadoon@utoronto.ca
Sameena Uddin suddin@cvh.on.ca
Anita Anand anita.anand@utoronto.ca
Anita B